In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataset

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-07 13:05:56.756 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /root/snn-encoder-test


In [3]:
import gc
import os

from loguru import logger
import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import optuna
from sklearn.model_selection import KFold, train_test_split
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler

from eeg_snn_encoder.encoders import PhaseEncoderExpand
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig

In [5]:
# Initialize the Optuna study
study = optuna.load_study(
    study_name="Classifier_PE_Tuning",
    storage=os.environ["OPTUNA_CONN_STRING"],
)

best_params = study.best_trial.params

model_params: ModelConfig = {
    "threshold": best_params["threshold"],
    "slope": best_params["slope"],
    "beta": best_params["beta"],
    "dropout_rate1": best_params["dropout_rate1"],
    "dropout_rate2": best_params["dropout_rate2"],
}

optimizer_params: OptimizerConfig = {
    "lr": best_params["lr"],
    "weight_decay": best_params["weight_decay"],
    "scheduler_factor": best_params["scheduler_factor"],
    "scheduler_patience": best_params["scheduler_patience"],
}

encoder_params = {
    "phase_window": best_params["phase_window"],
}

In [6]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

fold_results = []

for fold, (train_val_ids, test_ids) in enumerate(kfold.split(dataset)):
    logger.info(f"Starting fold {fold + 1} of {kfold.n_splits} BurstEncoder")

    train_ids, val_ids = train_test_split(
        train_val_ids, test_size=0.2, random_state=42, shuffle=True
    )

    train_sampler = SubsetRandomSampler(train_ids)
    val_sampler = SubsetRandomSampler(val_ids)
    test_sampler = SubsetRandomSampler(test_ids)

    trainloader = DataLoader(dataset, batch_size=512, sampler=train_sampler)
    valloader = DataLoader(dataset, batch_size=512, sampler=val_sampler)
    testloader = DataLoader(dataset, batch_size=512, sampler=test_sampler)

    spike_encoder = PhaseEncoderExpand(**encoder_params)

    lit_model = LitSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=50,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_checkpointing=False,
        callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=5)],
        logger=False,
    )

    trainer.fit(lit_model, trainloader, valloader)
    trainer.test(lit_model, testloader)

    test_loss = trainer.callback_metrics["test_loss"].item()
    test_acc = trainer.callback_metrics["test_acc"]
    test_precision = trainer.callback_metrics["test_precision"]
    test_recall = trainer.callback_metrics["test_recall"]
    test_f1 = trainer.callback_metrics["test_f1"]
    test_mse = trainer.callback_metrics["test_mse"]
    test_total_spikes = trainer.callback_metrics["test_total_spikes"]

    logger.info(
        f"Fold {fold + 1} - Test Loss: {test_loss:.4f}, "
        f"Test Accuracy: {test_acc:.4f}, "
        f"Test Precision: {test_precision:.4f}, "
        f"Test Recall: {test_recall:.4f}, "
        f"Test F1: {test_f1:.4f}, "
        f"Test MSE: {test_mse:.4f}, "
        f"Test Total Spikes: {test_total_spikes:.4f}"
    )

    fold_results.append(
        {
            "fold": fold,
            "test_loss": test_loss,
            "test_acc": test_acc,
            "test_precision": test_precision,
            "test_recall": test_recall,
            "test_f1": test_f1,
            "test_mse": test_mse,
            "test_total_spikes": test_total_spikes,
        }
    )

    del lit_model
    del trainer
    del spike_encoder
    torch.cuda.empty_cache()
    gc.collect()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type                   | Params | Mode 
---------------------------------------------------------
0 | model | EEGSTFTSpikeClassifier | 824 K  | train
---------------------------------------------------------
824 K     Trainable params
0         Non-trainable params
824 K     Total params
3.299     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode


2025-05-07 13:08:05.311 | INFO     | __main__:<module>:6 - Starting fold 1 of 5 BurstEncoder


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/root/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/root/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8065802454948425     │
│          test_f1          │    0.7948448061943054     │
│         test_loss         │     9.312493324279785     │
│         test_mse          │    0.19341973960399628    │
│      test_precision       │    0.8048060536384583     │
│        test_recall        │    0.7853487730026245     │
│     test_total_spikes     │      29625.630859375      │
└───────────────────────────┴───────────────────────────┘

2025-05-07 13:21:41.683 | INFO     | __main__:<module>:49 - Fold 5 - Test Loss: 9.3125, Test Accuracy: 0.8066, Test Precision: 0.8048, Test Recall: 0.7853, Test F1: 0.7948, Test MSE: 0.1934, Test Total Spikes: 29625.6309


In [7]:
import pandas as pd

results_df = pd.DataFrame(fold_results)
tensor_cols = results_df.select_dtypes(include=["object"]).columns

for col in tensor_cols:
    results_df[col] = results_df[col].apply(lambda x: x.item() if hasattr(x, "item") else x)

results_df.set_index("fold", inplace=True)
results_df

,test_loss,test_acc,test_precision,test_recall,test_f1,test_mse,test_total_spikes
fold,,,,,,,
0,11.052766,0.743028,0.721883,0.811693,0.764146,0.256972,29705.914062
1,21.901409,0.575697,0.547566,0.955035,0.695978,0.424303,30284.902344
2,9.306674,0.801793,0.774707,0.842168,0.806978,0.198207,30050.941406
3,9.477697,0.797607,0.790227,0.819620,0.804569,0.202393,29847.306641
4,9.312493,0.806580,0.804806,0.785349,0.794845,0.193420,29625.630859


In [9]:
from eeg_snn_encoder.config import REPORTS_DIR

results_df.to_csv(
    REPORTS_DIR / "pe_model_results.csv", index=True
)